# DACER
(trying to use DACER with metaworld, but it was originally used with gymnasium)
(wrote many suspicious wrappers)

# The following appears to work!!!

In [ ]:
import time
from types import SimpleNamespace
from pathlib import Path

import jax
import jax.numpy as jnp
import numpy as np
from metaworld import MT10
from gymnasium import ObservationWrapper, spaces
from gymnasium.core import Wrapper
import random
import subprocess
from relax.algorithm.dacer import DACER
from relax.buffer import TreeBuffer
from relax.network.dacer import create_dacer_net
from relax.trainer.off_policy import OffPolicyTrainer
from relax.utils.experience import Experience
from relax.utils.fs import PROJECT_ROOT
from relax.utils.random_utils import seeding

#probably can pick better hyperparameters...
seed=100
hidden_num= 3
hidden_dim =256
diffusion_steps= 20
diffusion_hidden_dim   = 256
start_step =int(1e3)  
total_step =int(1e6)
lr=1e-4


class OneHotTaskWrapper(ObservationWrapper):
    def __init__(self, env, env_idx, all_tasks, n_envs):
        super().__init__(env)
        self.env_idx= env_idx
        self.all_tasks= all_tasks
        self.n_envs=n_envs

        self.n_configs= len(all_tasks) // n_envs

        orig_low,orig_high=env.observation_space.low,env.observation_space.high
        low=np.concatenate([orig_low,np.zeros(n_envs,dtype=np.float32)])
        high=np.concatenate([orig_high,np.ones(n_envs,dtype=np.float32)])
        self.observation_space=spaces.Box(low=low,high=high,dtype=np.float32)

#sample a task and config within task upon each env restart. task is random
#can also restrict range of new tasks (<n for MTn) if you'd like
    def reset(self, **kwargs):
        cfg_idx=random.randrange(self.n_configs)
        task=self.all_tasks[self.env_idx*self.n_configs+cfg_idx]
        self.env.unwrapped.set_task(task)
        obs,info=self.env.reset(**kwargs)
        return self._append_one_hot(obs),info

    def observation(self,obs):
        return self._append_one_hot(obs)

    def _append_one_hot(self, obs):
        one_hot = np.zeros(self.n_envs,dtype=np.float32)
        one_hot[self.env_idx]=1.0
        return np.concatenate([obs, one_hot],axis=-1)

# trying to bypass errors w/ more wrappers (chat suggested) (honestly kind of suspicious, not sure what internal logic is being bypassed)
class SpecWrapper(Wrapper):
    def __init__(self, env, env_id: str):
        super().__init__(env)
        self._spec = SimpleNamespace(id=env_id)
    @property
    def spec(self):
        return self._spec

ml10= MT10()
all_tasks = ml10.train_tasks
n_envs= len(ml10.train_classes)

raw=ml10.train_classes[all_tasks[0].env_name]()
raw.seed(seed)
env=OneHotTaskWrapper(raw, all_tasks, n_envs)
env=SpecWrapper(env, "MT10")

#getting a lot of dimensionality issues
obs_dim=env.observation_space.shape[0]
act_dim=env.action_space.shape[0]
eval_env=None


master_rng, _ = seeding(seed)
env_seed,env_action_seed,buffer_seed, init_key_seed,train_key_seed = map(
    int,master_rng.integers(0, 2**32 - 1, 5)
)
init_key =jax.random.key(init_key_seed)
train_key=jax.random.key(train_key_seed)

buffer=TreeBuffer.from_experience(obs_dim, act_dim, size=int(1e6), seed=buffer_seed)

def mish(x: jnp.ndarray)->jnp.ndarray:
    return x*jnp.tanh(jax.nn.softplus(x))

hidden_sizes=[hidden_dim] * hidden_num
diffusion_sizes=[diffusion_hidden_dim] * hidden_num
agent,params=create_dacer_net(
    init_key,
    obs_dim,
    act_dim,
    hidden_sizes,
    diffusion_sizes,
    mish,
    num_timesteps=diffusion_steps
)
algorithm=DACER(agent, params, lr=lr)

#using instead of OffPolicyTrainer bc it keeps giving eval error
#tbh needs a whole rewrite
class NoEvalTrainer(OffPolicyTrainer):
    def setup(self, dummy_data):
        Path(self.log_path).mkdir(parents=True, exist_ok=True)
        orig_popen = subprocess.Popen
        subprocess.Popen=lambda *args, **kwargs: SimpleNamespace(
            stdin=SimpleNamespace(close=lambda: None),
            wait=lambda *args, **kwargs: None
        )
        try:
            super().setup(dummy_data)
        finally:
            subprocess.Popen = orig_popen

log_dir=PROJECT_ROOT/"logs"/f"MT10_dacer_{time.strftime('%Y-%m-%d_%H-%M-%S')}_s{seed}"

#now actually running DACER
trainer=NoEvalTrainer(
    env=env,
    algorithm=algorithm,
    buffer=buffer,
    start_step=start_step,
    total_step=total_step,
    sample_per_iteration=1,
    evaluate_env=eval_env,
    save_policy_every=300_000,
    warmup_with="random",
    log_path=log_dir,
)
trainer.setup(Experience.create_example(obs_dim, act_dim, trainer.batch_size))
trainer.run(train_key)

/home/saba/miniconda3/envs/relax/lib/python3.11/site-packages/gymnasium/spaces/box.py:235: UserWarning: WARN: Box low's precision lowered by casting to float32, current low.dtype=float64
  gym.logger.warn(
/home/saba/miniconda3/envs/relax/lib/python3.11/site-packages/gymnasium/spaces/box.py:305: UserWarning: WARN: Box high's precision lowered by casting to float32, current high.dtype=float64
  gym.logger.warn(


DistributionalQNet2(hidden_sizes=[256, 256, 256], activation=<function mish at 0x7fa122fef060>, output_activation=<function <lambda> at 0x7fa87ca11ee0>, name='distributional_q_net2')
DistributionalQNet2(hidden_sizes=[256, 256, 256], activation=<function mish at 0x7fa122fef060>, output_activation=<function <lambda> at 0x7fa87ca11ee0>, name='distributional_q_net2')
DACERPolicyNet(hidden_sizes=[256, 256, 256], activation=<function mish at 0x7fa122fef060>, output_activation=<function <lambda> at 0x7fa87ca13f60>, time_dim=16, name='dacer_policy_net')


'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring feature)
'+ptx89' is not a recognized feature for this target (ignoring f

In [ ]:
# semantics!
contextArray = np.array([
    [ 2.55721937e+00,  2.31504669e+00,  1.53086022e+00,  3.67148980e+00,
     -7.59686234e-01, -7.61520546e-01,  7.11087296e-01, -2.76179305e-01,
     -9.09933731e-03, -4.92187850e-16],
    [ 3.19798408e+00,  2.29742495e-01,  1.50240787e+00, -7.02709710e-01,
     -1.11196737e-01,  2.74924940e+00, -1.53620950e+00, -1.18464051e-01,
      3.12674877e-02, -4.92187850e-16],
    [ 3.28570139e+00,  1.92692870e+00,  2.16563078e-01, -3.17126572e+00,
      1.22878090e+00, -1.05485421e+00,  1.25747231e+00,  3.66652176e-01,
      2.40985359e-02, -4.92187850e-16],
    [-2.08496350e+00,  4.64285575e-01,  5.41231677e-01,  6.38912049e-01,
      2.08218306e+00, -1.46809404e+00, -2.13341308e+00,  7.39046881e-01,
     -2.00950098e-01, -4.92187850e-16],
    [ 4.27420596e-01, -8.70185256e-01, -2.86820046e+00,  3.71623910e-01,
      1.72775859e+00, -2.57396544e-02, -3.41897534e-02, -1.64198974e+00,
     -1.41869863e-01, -4.92187850e-16],
    [ 1.15915385e-01, -1.24388372e+00, -2.73867371e+00,  1.45164344e+00,
      5.18528498e-01,  1.21674038e+00,  9.57762804e-01,  1.37271833e+00,
      3.60078564e-01, -4.92187850e-16],
    [ 1.14941507e+00, -4.93749060e+00,  2.04079053e+00, -2.26013967e-01,
     -7.33097844e-01, -1.07109692e+00,  3.29521154e-01, -4.60878162e-02,
     -9.44210158e-03, -4.92187850e-16],
    [-1.88857968e-01,  8.08267667e-01, -2.38279611e+00, -1.02331244e+00,
     -3.41088830e+00, -9.48025784e-01, -1.05931369e+00,  2.31463462e-02,
      9.69085706e-02, -4.92187850e-16],
    [-4.71406517e+00,  8.02744073e-01,  1.52039680e+00, -5.83708654e-01,
      3.05733353e-02,  4.24301726e-01,  5.31541815e-01, -4.67244928e-01,
      1.12817178e+00, -4.92187850e-16],
    [-3.74576926e+00,  5.04544376e-01,  6.37420105e-01, -4.26658712e-01,
     -5.72955276e-01,  9.39039647e-01,  9.75740641e-01,  4.84021130e-02,
     -1.27916354e+00, -4.92187850e-16]
], dtype=float)

In [ ]:
import time
from types import SimpleNamespace
from pathlib import Path

import jax
import jax.numpy as jnp
import numpy as np
from metaworld import MT10
from gymnasium import ObservationWrapper, spaces
from gymnasium.core import Wrapper
import random
import subprocess
from relax.algorithm.dacer import DACER
from relax.buffer import TreeBuffer
from relax.network.dacer import create_dacer_net
from relax.trainer.off_policy import OffPolicyTrainer
from relax.utils.experience import Experience
from relax.utils.fs import PROJECT_ROOT
from relax.utils.random_utils import seeding

#probably can pick better hyperparameters...
seed=100
hidden_num= 3
hidden_dim =256
diffusion_steps= 20
diffusion_hidden_dim   = 256
start_step =int(1e3)  
total_step =int(1e6)
lr=1e-4


class OneHotTaskWrapper(ObservationWrapper):
    def __init__(self, env, env_idx, all_tasks, n_envs):
        super().__init__(env)
        self.env_idx= env_idx
        self.all_tasks= all_tasks
        self.n_envs=n_envs

        self.n_configs= len(all_tasks) // n_envs

        orig_low,orig_high=env.observation_space.low,env.observation_space.high
        low=np.concatenate([orig_low,np.zeros(n_envs,dtype=np.float32)])
        high=np.concatenate([orig_high,np.ones(n_envs,dtype=np.float32)])
        self.observation_space=spaces.Box(low=low,high=high,dtype=np.float32)

#sample a task and config within task upon each env restart. task is random
#can also restrict range of new tasks (<n for MTn) if you'd like
    def reset(self, **kwargs):
        cfg_idx=random.randrange(self.n_configs)
        task=self.all_tasks[self.env_idx*self.n_configs+cfg_idx]
        self.env.unwrapped.set_task(task)
        obs,info=self.env.reset(**kwargs)
        return self._append_one_hot(obs),info

    def observation(self,obs):
        return self._append_one_hot(obs)

    def _append_one_hot(self, obs):
        one_hot = np.zeros(self.n_envs, dtype=np.float32)
        one_hot[self.env_idx] = 1.0
        #to get the semantic embedding from semantic matrix + onehot
        #use jax here in the future
        one_hot_mod=np.einsum('ik,kj->ij', np.array([one_hot]),contextArray)[0]
        return np.concatenate([obs, one_hot_mod], axis=-1)

# trying to bypass errors w/ more wrappers (chat suggested) (honestly kind of suspicious, not sure what internal logic is being bypassed)
class SpecWrapper(Wrapper):
    def __init__(self, env, env_id: str):
        super().__init__(env)
        self._spec = SimpleNamespace(id=env_id)
    @property
    def spec(self):
        return self._spec

ml10= MT10()
all_tasks = ml10.train_tasks
n_envs= len(ml10.train_classes)

raw=ml10.train_classes[all_tasks[0].env_name]()
raw.seed(seed)
env=OneHotTaskWrapper(raw, all_tasks, n_envs)
env=SpecWrapper(env, "MT10")

#getting a lot of dimensionality issues
obs_dim=env.observation_space.shape[0]
act_dim=env.action_space.shape[0]
eval_env=None


master_rng, _ = seeding(seed)
env_seed,env_action_seed,buffer_seed, init_key_seed,train_key_seed = map(
    int,master_rng.integers(0, 2**32 - 1, 5)
)
init_key =jax.random.key(init_key_seed)
train_key=jax.random.key(train_key_seed)

buffer=TreeBuffer.from_experience(obs_dim, act_dim, size=int(1e6), seed=buffer_seed)

def mish(x: jnp.ndarray)->jnp.ndarray:
    return x*jnp.tanh(jax.nn.softplus(x))

hidden_sizes=[hidden_dim] * hidden_num
diffusion_sizes=[diffusion_hidden_dim] * hidden_num
agent,params=create_dacer_net(
    init_key,
    obs_dim,
    act_dim,
    hidden_sizes,
    diffusion_sizes,
    mish,
    num_timesteps=diffusion_steps
)
algorithm=DACER(agent, params, lr=lr)

#using instead of OffPolicyTrainer bc it keeps giving eval error
#tbh needs a whole rewrite
class NoEvalTrainer(OffPolicyTrainer):
    def setup(self, dummy_data):
        Path(self.log_path).mkdir(parents=True, exist_ok=True)
        orig_popen = subprocess.Popen
        subprocess.Popen=lambda *args, **kwargs: SimpleNamespace(
            stdin=SimpleNamespace(close=lambda: None),
            wait=lambda *args, **kwargs: None
        )
        try:
            super().setup(dummy_data)
        finally:
            subprocess.Popen = orig_popen

log_dir=PROJECT_ROOT/"logs"/f"MT10_dacer_{time.strftime('%Y-%m-%d_%H-%M-%S')}_s{seed}"

#now actually running DACER
trainer=NoEvalTrainer(
    env=env,
    algorithm=algorithm,
    buffer=buffer,
    start_step=start_step,
    total_step=total_step,
    sample_per_iteration=1,
    evaluate_env=eval_env,
    save_policy_every=300_000,
    warmup_with="random",
    log_path=log_dir,
)
trainer.setup(Experience.create_example(obs_dim, act_dim, trainer.batch_size))
trainer.run(train_key)